# Running the simple RL notebook

In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile]-s, pile

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n:int, _a:str, _b:str)->(int,int):
    from collections import defaultdict
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    # info
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

In [4]:
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random  509    Random  491
1000 games,     Guru 1000    Random    0
1000 games,   Random    8      Guru  992
1000 games,     Guru  929      Guru   71


In [5]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n:int):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n games
    for _ in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            
            # Switch sides for play and learning
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [6]:
%%time

nim_qlearn(1000)

CPU times: user 44.6 ms, sys: 2.76 ms, total: 47.3 ms
Wall time: 46.3 ms


In [7]:
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')

play_games(1000, 'Random', 'Random') ;

1000 games, Qlearner  722    Random  278
1000 games,   Random  293  Qlearner  707
1000 games,   Random  500    Random  500


In [8]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
Wins = []
for n in n_train:
    nim_qlearn(n)
    wins_a, wins_b = play_games(1000, 'Qlearner', 'Random')
    Wins += [wins_a/(wins_a+wins_b)]

1000 games, Qlearner  521    Random  479
1000 games, Qlearner  589    Random  411
1000 games, Qlearner  647    Random  353
1000 games, Qlearner  721    Random  279
1000 games, Qlearner  733    Random  267
1000 games, Qlearner  703    Random  297
1000 games, Qlearner  724    Random  276
CPU times: user 3.35 s, sys: 20 ms, total: 3.37 s
Wall time: 3.36 s


In [9]:
print(Wins)


[0.521, 0.589, 0.647, 0.721, 0.733, 0.703, 0.724]


In [10]:
# Function to print the entire set of states
def qtable_log(_fn:str):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable_log('qtable_debug.txt')